In [41]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jun  1 17:31:40 2020

@author: manav

Modifed on 23 AUG 2022

by mahayasa adiputra
"""

import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from imblearn.over_sampling import SMOTE
import sklearn.metrics as mt
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import NeighbourhoodCleaningRule
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from numpy import mean
from numpy import std
from imblearn.combine import SMOTEENN
import time
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


# record start time
start = time.time()
le = LabelEncoder()
# Importing the dataset
dataset=pd.read_csv('https://raw.githubusercontent.com/mahayasa/various-sampling-churn-prediction/main/data/telco_churn.csv')

#transform data into numeric value
dataset['Partner']=le.fit_transform(dataset['Partner'])
dataset['Dependents']=le.fit_transform(dataset['Dependents'])
dataset['PhoneService']=le.fit_transform(dataset['PhoneService'])
dataset['MultipleLines']=le.fit_transform(dataset['MultipleLines'])
dataset['InternetService']=le.fit_transform(dataset['InternetService'])
dataset['OnlineSecurity']=le.fit_transform(dataset['OnlineSecurity'])
dataset['DeviceProtection']=le.fit_transform(dataset['DeviceProtection'])
dataset['TechSupport']=le.fit_transform(dataset['TechSupport'])
dataset['StreamingTV']=le.fit_transform(dataset['StreamingTV'])
dataset['StreamingMovies']=le.fit_transform(dataset['StreamingMovies'])
dataset['Contract']=le.fit_transform(dataset['Contract'])
dataset['PaperlessBilling']=le.fit_transform(dataset['PaperlessBilling'])
dataset['PaymentMethod']=le.fit_transform(dataset['PaymentMethod'])
dataset['gender']=le.fit_transform(dataset['gender'])
dataset['OnlineBackup']=le.fit_transform(dataset['OnlineBackup'])

#filling missing value with mean
dataset['TotalCharges'].replace(to_replace = 0, value = dataset['TotalCharges'].mean(), inplace=True)
X=dataset.drop(['Churn','customerID'],axis=1)
y=dataset["Churn"]
y = le.fit_transform(y)
#imblearn SMOTE
#smote_enn = SMOTE()
smote_enn = SMOTEENN()
X, y = smote_enn.fit_resample(X, y)
#smote_enn = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
#X, y = smote_enn.fit_resample(X, y)
#ncr = NeighbourhoodCleaningRule(n_neighbors=5, kind_sel='all')
#X, y = ncr.fit_resample(X, y)

#kfold cross validation
cv = KFold(n_splits=5, random_state=1, shuffle=True)

end = time.time()

from google.colab import drive
drive.mount('/content/drive')

print("The time of execution of preprocess:",
      (end-start), "s")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
The time of execution of preprocess: 2.787808656692505 s


In [51]:
# decision tree
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-tn-telco1-dt-cs-auc.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/smoteenn-tn-telco1-dt-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco1"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.937189 using {'class_weight': {0: 100, 1: 90}}
0.932359 (0.006981) with: {'class_weight': {0: 100, 1: 10}}
0.936680 (0.010097) with: {'class_weight': {0: 100, 1: 20}}
0.935878 (0.006931) with: {'class_weight': {0: 100, 1: 30}}
0.934412 (0.009562) with: {'class_weight': {0: 100, 1: 40}}
0.933965 (0.008586) with: {'class_weight': {0: 100, 1: 50}}
0.933969 (0.008288) with: {'class_weight': {0: 100, 1: 60}}
0.933722 (0.009573) with: {'class_weight': {0: 100, 1: 70}}
0.934136 (0.008480) with: {'class_weight': {0: 100, 1: 80}}
0.937189 (0.008658) with: {'class_weight': {0: 100, 1: 90}}
0.936428 (0.007910) with: {'class_weight': {0: 100, 1: 100}}


In [52]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-tn-telco1-dt-cs-f1.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/smoteenn-tn-telco1-dt-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco1"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.937169 using {'class_weight': {0: 100, 1: 30}}
0.932736 (0.007203) with: {'class_weight': {0: 100, 1: 10}}
0.936559 (0.009757) with: {'class_weight': {0: 100, 1: 20}}
0.937169 (0.008259) with: {'class_weight': {0: 100, 1: 30}}
0.934283 (0.009396) with: {'class_weight': {0: 100, 1: 40}}
0.934644 (0.010772) with: {'class_weight': {0: 100, 1: 50}}
0.933258 (0.009200) with: {'class_weight': {0: 100, 1: 60}}
0.935319 (0.009485) with: {'class_weight': {0: 100, 1: 70}}
0.936281 (0.009591) with: {'class_weight': {0: 100, 1: 80}}
0.936754 (0.009293) with: {'class_weight': {0: 100, 1: 90}}
0.935188 (0.007370) with: {'class_weight': {0: 100, 1: 100}}


In [53]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-tn-telco1-dt-cs-gmean.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/smoteenn-tn-telco1-dt-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco1"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.936789 using {'class_weight': {0: 100, 1: 100}}
0.932193 (0.007226) with: {'class_weight': {0: 100, 1: 10}}
0.935258 (0.009446) with: {'class_weight': {0: 100, 1: 20}}
0.934991 (0.006120) with: {'class_weight': {0: 100, 1: 30}}
0.931999 (0.011215) with: {'class_weight': {0: 100, 1: 40}}
0.934187 (0.009105) with: {'class_weight': {0: 100, 1: 50}}
0.932513 (0.008456) with: {'class_weight': {0: 100, 1: 60}}
0.933549 (0.009885) with: {'class_weight': {0: 100, 1: 70}}
0.935574 (0.007951) with: {'class_weight': {0: 100, 1: 80}}
0.936179 (0.008804) with: {'class_weight': {0: 100, 1: 90}}
0.936789 (0.006910) with: {'class_weight': {0: 100, 1: 100}}


In [54]:
# LR
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-tn-telco1-lr-cs-auc.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/smoteenn-tn-telco1-lr-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco1"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.979130 using {'class_weight': {0: 100, 1: 10}}
0.979130 (0.004159) with: {'class_weight': {0: 100, 1: 10}}
0.978748 (0.004124) with: {'class_weight': {0: 100, 1: 20}}
0.978626 (0.003955) with: {'class_weight': {0: 100, 1: 30}}
0.978018 (0.003948) with: {'class_weight': {0: 100, 1: 40}}
0.978616 (0.004338) with: {'class_weight': {0: 100, 1: 50}}
0.978453 (0.004134) with: {'class_weight': {0: 100, 1: 60}}
0.978591 (0.004366) with: {'class_weight': {0: 100, 1: 70}}
0.978362 (0.004360) with: {'class_weight': {0: 100, 1: 80}}
0.977870 (0.004201) with: {'class_weight': {0: 100, 1: 90}}
0.978097 (0.004667) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [55]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-tn-telco1-lr-cs-f1.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/smoteenn-tn-telco1-lr-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco1"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.926322 using {'class_weight': {0: 100, 1: 100}}
0.867291 (0.007321) with: {'class_weight': {0: 100, 1: 10}}
0.896526 (0.010004) with: {'class_weight': {0: 100, 1: 20}}
0.905497 (0.008366) with: {'class_weight': {0: 100, 1: 30}}
0.909792 (0.009209) with: {'class_weight': {0: 100, 1: 40}}
0.913499 (0.010803) with: {'class_weight': {0: 100, 1: 50}}
0.917776 (0.009181) with: {'class_weight': {0: 100, 1: 60}}
0.919756 (0.010479) with: {'class_weight': {0: 100, 1: 70}}
0.922727 (0.008840) with: {'class_weight': {0: 100, 1: 80}}
0.924688 (0.009452) with: {'class_weight': {0: 100, 1: 90}}
0.926322 (0.009685) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [56]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-tn-telco1-lr-cs-gmean.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/smoteenn-telco1-lr-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco1"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.926176 using {'class_weight': {0: 100, 1: 100}}
0.872384 (0.007429) with: {'class_weight': {0: 100, 1: 10}}
0.901245 (0.009843) with: {'class_weight': {0: 100, 1: 20}}
0.909467 (0.008112) with: {'class_weight': {0: 100, 1: 30}}
0.913053 (0.008803) with: {'class_weight': {0: 100, 1: 40}}
0.916201 (0.010432) with: {'class_weight': {0: 100, 1: 50}}
0.919880 (0.008852) with: {'class_weight': {0: 100, 1: 60}}
0.921398 (0.009964) with: {'class_weight': {0: 100, 1: 70}}
0.923718 (0.008427) with: {'class_weight': {0: 100, 1: 80}}
0.925129 (0.009425) with: {'class_weight': {0: 100, 1: 90}}
0.926176 (0.009388) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [57]:
#SVM
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
model = SVC()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-tn-telco1-svm-cs-auc.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/smoteenn-tn-telco1-svm-cs-auc.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco1"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.963956 using {'class_weight': {0: 100, 1: 60}}
0.949526 (0.005967) with: {'class_weight': {0: 100, 1: 10}}
0.960687 (0.006326) with: {'class_weight': {0: 100, 1: 20}}
0.962146 (0.005590) with: {'class_weight': {0: 100, 1: 30}}
0.963449 (0.005163) with: {'class_weight': {0: 100, 1: 40}}
0.963855 (0.004906) with: {'class_weight': {0: 100, 1: 50}}
0.963956 (0.004787) with: {'class_weight': {0: 100, 1: 60}}
0.963797 (0.004709) with: {'class_weight': {0: 100, 1: 70}}
0.963745 (0.004639) with: {'class_weight': {0: 100, 1: 80}}
0.963722 (0.004611) with: {'class_weight': {0: 100, 1: 90}}
0.963825 (0.004649) with: {'class_weight': {0: 100, 1: 100}}


In [58]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-tn-telco1-svm-cs-f1.xlsx', index=False)

# Define the source file path (in Colab)
source_file = '/content/smoteenn-tn-telco1-svm-cs-f1.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco1"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.899388 using {'class_weight': {0: 100, 1: 70}}
0.760934 (0.014533) with: {'class_weight': {0: 100, 1: 10}}
0.823083 (0.010698) with: {'class_weight': {0: 100, 1: 20}}
0.850851 (0.011011) with: {'class_weight': {0: 100, 1: 30}}
0.864698 (0.010664) with: {'class_weight': {0: 100, 1: 40}}
0.878664 (0.009099) with: {'class_weight': {0: 100, 1: 50}}
0.893288 (0.011298) with: {'class_weight': {0: 100, 1: 60}}
0.899388 (0.009035) with: {'class_weight': {0: 100, 1: 70}}
0.898470 (0.008297) with: {'class_weight': {0: 100, 1: 80}}
0.894341 (0.007884) with: {'class_weight': {0: 100, 1: 90}}
0.892806 (0.008219) with: {'class_weight': {0: 100, 1: 100}}


In [59]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('smoteenn-tn-telco1-svm-cs-gmean.xlsx', index=False)


# Define the source file path (in Colab)
source_file = '/content/smoteenn-tn-telco1-svm-cs-gmean.xlsx'

# Define the destination folder path in Google Drive
destination_folder = "/content/drive/MyDrive/experiment/telco1"

# Copy the file to the destination
!cp $source_file $destination_folder

Best: 0.901404 using {'class_weight': {0: 100, 1: 70}}
0.759882 (0.015789) with: {'class_weight': {0: 100, 1: 10}}
0.826944 (0.011118) with: {'class_weight': {0: 100, 1: 20}}
0.855826 (0.011119) with: {'class_weight': {0: 100, 1: 30}}
0.869621 (0.010641) with: {'class_weight': {0: 100, 1: 40}}
0.883111 (0.009018) with: {'class_weight': {0: 100, 1: 50}}
0.896741 (0.011075) with: {'class_weight': {0: 100, 1: 60}}
0.901404 (0.008951) with: {'class_weight': {0: 100, 1: 70}}
0.899446 (0.008346) with: {'class_weight': {0: 100, 1: 80}}
0.894291 (0.008028) with: {'class_weight': {0: 100, 1: 90}}
0.892081 (0.008328) with: {'class_weight': {0: 100, 1: 100}}
